In [ ]:
# Copyright 2019 TerraPower, LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
GUI elements for manipulating grid layout and contents.

This provides a handful of classes which provide wxPython Controls for manipulating
grids and grid Blueprints.


**Known Issues**

* There is no action stack or undo functionality. Save frequently if you want to
  recover previous states

* Cartesian grids are supported, but not rendered as nicely as their Hex counterparts.
  The "through center assembly" case is not rendered properly with the half-assemblies
  that lie along the edges.

* The controls are optimized for manipulating a Core layout, displaying an "Assembly
  palette" that contains the Assembly designs found in the top-level blueprints. A little
  extra work and this could also be made to manipulate block grids or other things.

* Assembly colors are derived from the set of flags applied to them, but the mapping of
  colors to flags is not particularly rich, and there isn't anything to disambiguate
  between asemblies of different design, but the same flags.

* No proper zoom support, and object sizes are fixed and don't accommodate long
  specifiers. Adding zoom would make for a fun first task to a new developer interested
  in computer graphics.

"""
import colorsys
import copy
from dataclasses import dataclass
import enum
import io
import os
import pathlib
import sys
from typing import Dict, Optional, Sequence, Tuple, Union

import numpy
import numpy.linalg

import armi
armi.configure(permissive=True)

from armi import runLog
from armi.utils import hexagon
from armi.utils import textProcessors
from armi.utils import asciimaps
from armi.settings.caseSettings import Settings
from armi.reactor import geometry
from armi.reactor import grids
from armi.reactor import blueprints
from armi.reactor.flags import Flags
import armi.reactor.blueprints
from armi.reactor.blueprints import Blueprints, gridBlueprint, migrate
from armi.reactor.blueprints.gridBlueprint import GridBlueprint, saveToStream
from armi.reactor.blueprints.assemblyBlueprint import AssemblyBlueprint
from armi.settings.fwSettings import globalSettings


UNIT_SIZE = 50  # pixels per assembly
UNIT_MARGIN = 40  # offset applied to the draw area margins

# The color to use for each object is based on the flags that that object has. All
# applicable colors will be blended together to produce the final color for the object.
# There are also plans to apply brush styles like cross-hatching or the like, which is
# what the Nones are for below. Future work to employ these. Colors are RGB fractions.
FLAG_STYLES = {
    # Red
    Flags.FUEL: (numpy.array([1.0, 0.0, 0.0]), None),
    # Green
    Flags.CONTROL: (numpy.array([0.0, 1.0, 0.0]), None),
    # Gray
    Flags.SHIELD: (numpy.array([0.4, 0.4, 0.4]), None),
    # Yellow
    Flags.REFLECTOR: (numpy.array([0.5, 0.5, 0.0]), None),
    # Paisley?
    Flags.INNER: (numpy.array([0.5, 0.5, 1.0]), None),
    # We shouldn't see many SECONDARY, OUTER, MIDDLE, etc. on their own, so these
    # will just darken or brighten whatever color we would otherwise get)
    Flags.SECONDARY: (numpy.array([0.0, 0.0, 0.0]), None),
    Flags.OUTER: (numpy.array([0.0, 0.0, 0.0]), None),
    # WHITE (same as above, this will just lighten anything that it accompanies)
    Flags.MIDDLE: (numpy.array([1.0, 1.0, 1.0]), None),
    Flags.ANNULAR: (numpy.array([1.0, 1.0, 1.0]), None),
}

# RGB weights for calculating luminance. We use this to decide whether we should put
# white or black text on top of the color. These come from CCIR 601
LUMINANCE_WEIGHTS = numpy.array([0.3, 0.59, 0.11])


@dataclass
class Point:
    x: int
    y: int


@dataclass
class Rect:
    topLeft: Point
    bottomRight: Point


@dataclass
class Color:
    r: int
    g: int
    b: int
    a: int


@dataclass
class Shape:
    pen_color: Color
    brush_color: Color
    polygon: Sequence[numpy.ndarray]
    label: str
    bounding_box: Rect
    


def _translationMatrix(x, y):
    """
    Return an affine transformation matrix representing an x- and y-translation.
    """
    return numpy.array([[1.0, 0.0, x], [0.0, 1.0, y], [0.0, 0.0, 1.0]])


def _boundingBox(points: Sequence[numpy.ndarray]) -> Rect:
    """
    Return the smallest wx.Rect that contains all of the passed points.
    """
    xmin = numpy.amin([p[0] for p in points])
    xmax = numpy.amax([p[0] for p in points])

    ymin = numpy.amin([p[1] for p in points])
    ymax = numpy.amax([p[1] for p in points])

    return Rect(Point(xmin, ymin), Point(xmax, ymax))


def _desaturate(c: Sequence[float]):
    r, g, b = tuple(c)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    l = l + (1.0 - l) * 0.5
    return numpy.array(colorsys.hls_to_rgb(h, l, s))


def _getColorAndBrushFromFlags(f, bold=True) -> Tuple[Color, Color]:
    """
    Given a set of Flags, return a wx.Pen and wx.Brush with which to draw a shape
    """
    c = numpy.array([0.0, 0.0, 0.0])
    nColors = 0

    for styleFlag, style in FLAG_STYLES.items():
        if not styleFlag & f:
            continue

        color, brush = style
        if color is not None:
            c += color
            nColors += 1
    if nColors:
        c /= nColors

    if not bold:
        # round-trip the rgb color through hsv so that we can desaturate
        c = _desaturate(c)

    luminance = c.dot(LUMINANCE_WEIGHTS)
    dark = luminance < 0.5

    c = tuple(int(255 * ci) for ci in c)
    
    pen_color = Color(255, 255, 255, 255) if dark else Color(0, 0, 0, 255)
    brush_color = Color(*c, 255)

    return pen_color, brush_color
#     brush = wx.Brush(wx.Colour(*c, 255))
#     pen = wx.WHITE if dark else wx.BLACK

#     return pen, brush


def _getShape(
    geom: geometry.GeomType,
    view: numpy.ndarray,
    model: Optional[numpy.ndarray] = None,
    label: str = "",
    description: Optional[str] = None,
    bold: bool = True,
) -> Shape:
    """
    Return a Shape to draw, given its GeomType and other relevant information.

    Parameters
    ----------
    geom: geometry.GeomType
        The geometry type, which defines the shape to be drawn
    view: numpy.ndarray
        A 3x3 matrix defining the world transform
    model: numpy.ndarray, optional
        A 3x3 matrix defining the model transform. No transform is made to the "unit"
        shape if no model transform is provided.
    label: str, optional
        A string label to draw on the shape
    description: str, optional
        A string containing metadata for determining how to style to shape
    bold: bool, optional
        Whether the object should be drawn with full saturation. Default ``True``
    """
    pen_color = Color(0, 0, 0, 255)
    brush_color = Color(200, 200, 200, 255)
    if description:
        aFlags = Flags.fromStringIgnoreErrors(description)
        pen_color, brush_color = _getColorAndBrushFromFlags(aFlags, bold=bold)

    if geom == geometry.GeomType.HEX:
        primitive = hexagon.corners(rotation=0)
    elif geom == geometry.GeomType.CARTESIAN:
        primitive = [(-0.5, -0.5), (0.5, -0.5), (0.5, 0.5), (-0.5, 0.5)]
    else:
        raise ValueError("Geom type `{}` unsupported".format(geom))

    # Appending 1 to each coordinate since the transformation matrix is 3x3
    poly = numpy.array([numpy.append(vertex, 1) for vertex in primitive]).transpose()
    model = model if model is not None else numpy.eye(3)
    poly = view.dot(model).dot(poly).transpose()
    poly = [vertex[0:2] for vertex in poly]

    boundingBox = _boundingBox(poly)

#     dc.SetTextForeground(color)
#     dc.DrawPolygon(poly)
#     dc.DrawLabel(label, boundingBox, wx.ALIGN_CENTRE)
    
    return Shape(pen_color, brush_color, poly, label, bounding_box)

#     return boundingBox

In [ ]:
from notebookjs import execute_js

with open("./grid_gui_lib.js", "r") as f:
    grid_gui_lib = f.read()

library_list = [
    "https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js",
    "https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.16.0/umd/popper.min.js",
    "https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js",
    "https://d3js.org/d3.v3.min.js",
] + [grid_gui_lib]

with open("./grid_gui.css", "r") as f:
    grid_gui_css = f.read()

css_list = [
    "https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css",
] + [grid_gui_css]

def update_graph(data):
    return {"results":data}

callbacks = {
    "update_graph": update_graph
}


In [ ]:
from armi.utils import asciimaps

asciimaps.HelloWorld()



In [ ]:
# A simple grid, drawn using svg and d3js.

execute_js(library_list=library_list,
           main_function="main",
           callbacks=callbacks,
           data_dict={
               "points": [
                   {"x_pos":0, "y_pos":0, "x_dat":0, "y_dat":0},
                   {"x_pos":0, "y_pos":1, "x_dat":0, "y_dat":0},
                   {"x_pos":1, "y_pos":0, "x_dat":0, "y_dat":0},
                   {"x_pos":1, "y_pos":1, "x_dat":0, "y_dat":0},
                ]},
           css_list=css_list)